In [1]:
import pandas as pd

In [2]:
ah = pd.read_excel('ds/Asentamiento Histo.xlsx')
lin = pd.read_excel('ds/LIN.xlsx')

In [3]:
ah = ah.drop(['Unnamed: 5', 'clave_autodenom', 'Unnamed: 7',
       'nom_autodenom_2008', 'nom_autodenom_1actualizacion',
       'Trans_fonetica_2008', 'Trans_fonetica_1actualización',
       'clave_ent_2000', 'clave_ent_2010', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'loc2015', 'tieneTocayas2000',
       'bandera2000', 'mayorHLI2000', 'HLI_2000', 'HLI_2005', 'HLI_2010',
       'edomunloc2000', 'edomunloc2010', 'clave_unica2000', 'clave_unica2010',
       'clave_unica2015', 'NDRIESGO', 'cambios_DI',
       'Claves para marcas de densidad ', 'Notas', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44'],axis=1)

In [4]:
ah = ah.dropna()

In [21]:
lin.columns

Index(['Lengua indígena', 'Entidad federativa', 'Municipio', 'Localidad',
       'AH/M', 'Población de 3 años y más que habla lengua indígena',
       'Hombres', 'Mujeres', 'AH/M.1'],
      dtype='object')

In [18]:
temporal = lin[['AH/M', 'Población de 3 años y más que habla lengua indígena',
       'Hombres', 'Mujeres', 'AH/M.1']]

In [19]:
temporal.head()

,AH/M,Población de 3 años y más que habla lengua indígena,Hombres,Mujeres,AH/M.1
0,NaN,1,1,0,NaN
1,NaN,1,1,0,NaN
2,NaN,1,1,0,NaN
3,NaN,2,1,1,NaN
4,NaN,2,1,1,NaN


In [22]:
#df2.set_index('player').subtract(df1.set_index('player'))

diff_df = pd.merge(lin[['Lengua indígena', 'Entidad federativa', 'Municipio', 'Localidad']], ah, how='outer', indicator='Exist')

In [23]:
diff_df = pd.concat([diff_df,temporal], axis=1)

In [24]:
diff_df.head()

,Lengua indígena,Entidad federativa,Municipio,Localidad,AH/M,Exist,AH/M,Población de 3 años y más que habla lengua indígena,Hombres,Mujeres,AH/M.1
0,Akateko,Aguascalientes,Aguascalientes,Aguascalientes,NaN,left_only,NaN,1,1,0,NaN
1,Akateko,Aguascalientes,Aguascalientes,Total,NaN,left_only,NaN,1,1,0,NaN
2,Akateko,Aguascalientes,Total,Total,NaN,left_only,NaN,1,1,0,NaN
3,Akateko,Baja California,Ensenada,Rodolfo Sánchez Taboada (Maneadero),NaN,left_only,NaN,2,1,1,NaN
4,Akateko,Baja California,Ensenada,Total,NaN,left_only,NaN,2,1,1,NaN


In [25]:
diff_df.to_csv("out/resultado.csv")